<a href="https://colab.research.google.com/github/JiaWeiXie/ntub_ai_bi/blob/main/ntub_bi_20220517.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
! wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz \
  && sudo tar -xzf ta-lib-0.4.0-src.tar.gz \
  && sudo rm ta-lib-0.4.0-src.tar.gz \
  && cd ta-lib/ \
  && sudo ./configure --prefix=/usr \
  && sudo make \
  && sudo make install \
  && cd ~ \
  && sudo rm -rf ta-lib/ \
  && pip install -U yfinance backtrader ta-lib

In [132]:
import os
import sys
import math

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.analyzers as btanalyzers

In [133]:
import yfinance as yf

spy = yf.Ticker("SPY")
dataset = spy.history(period="max")
# dataset.to_csv("spy_max.csv")
dataset["2019"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,233.141438,238.098477,233.113005,237.122223,126925200,0.0,0
2019-01-03,235.274024,235.596289,230.952028,231.463852,144140700,0.0,0
2019-01-04,234.667414,239.899310,234.269337,239.216888,142628800,0.0,0
2019-01-07,239.501232,242.591076,238.553425,241.103027,103139100,0.0,0
2019-01-08,243.415648,243.880064,240.742827,243.368240,102512600,0.0,0
...,...,...,...,...,...,...,...
2019-12-24,310.460928,310.509204,309.910441,310.229156,20270000,0.0,0
2019-12-26,310.634784,311.890282,310.625147,311.880615,30911200,0.0,0
2019-12-27,312.653182,312.711125,311.243190,311.803314,42528800,0.0,0


In [140]:
# sma cross strategy
class SmaCross(bt.Strategy):
    # 交易紀錄
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt}: {txt}")
    
    # 設定交易參數
    params = dict(
        ma_period_short=5,
        ma_period_long=10,
    )

    def __init__(self):
        # 均線交叉策略
        sma_5 = bt.ind.SimpleMovingAverage(
            self.data.close,
            period=self.p.ma_period_short,
          )
        sma_10 = bt.ind.SimpleMovingAverage(
            self.data.close,
            period=self.p.ma_period_long,
          )
        self.crossover = bt.ind.CrossOver(sma_5, sma_10)

        # 用開盤價做交易
        self.dataopen = self.datas[0].open

        self.ma_diff = sma_5 - sma_10

    def next(self):
      if len(list(self.ma_diff)) == 1 and self.position:
        self.close(price=self.dataopen[0])
        self.log("sell")
        return

      # 帳戶沒有部位
      if not self.position and self.ma_diff[0] < 0 and self.ma_diff[1] > 0:
          self.log("buy", self.datas[0].datetime.date(1))
          self.buy(price=self.dataopen[1])
      elif self.position and self.ma_diff[0] > 0 and self.ma_diff[1] < 0:
          self.log("sell", self.datas[0].datetime.date(1))
          self.close(price=self.dataopen[1])

In [141]:
data = bt.feeds.PandasData(dataname=dataset["2019"])

# 初始化cerebro
cerebro = bt.Cerebro()
# feed data
cerebro.adddata(data)
# add strategy
cerebro.addstrategy(SmaCross)
# All In
cerebro.addsizer(bt.sizers.AllInSizer)
# add analyzer
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
# run backtest
thestrats = cerebro.run()
thestrat = thestrats[0]

2019-03-14: buy
2019-04-01: buy
2019-05-21: buy
2019-06-07: buy
2019-07-02: buy
2019-07-22: sell
2019-07-26: buy
2019-08-14: buy
2019-08-15: sell
2019-08-22: buy
2019-09-03: buy
2019-09-24: sell
2019-10-14: buy
2019-12-05: sell
2019-12-10: buy
2019-12-31: sell


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


In [136]:
print("Returns:")

for k, v in thestrat.analyzers.returns.get_analysis().items():
  print(f"{k}:", f"{v * 100:.2f}%")

Returns:
rtot: 11.39%
ravg: 0.05%
rnorm: 12.06%
rnorm100: 1206.26%


In [137]:
cerebro.plot()

[[<Figure size 432x288 with 5 Axes>]]